<a href="https://colab.research.google.com/github/ubern-mia/bme-labs/session01/ixi-dataset-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Example

This example gives a brief example on how to do a simple classification from MRI data. It takes images from the IXI dataset, where we have information about the sex of the subject alongside an MRI. We will try to directly infer the sex of a given patient from MRI. It is heavily inspired by this MONAI example: https://colab.research.google.com/github/Project-MONAI/MONAIBootcamp2021/blob/master/day1/3.%20End-To-End%20Workflow%20with%20MONAI.ipynb 
MONAI is a framework built on PyTorch specifically for deep learning with medical imaging data.

To make the data easier to handle, the data is resized to a volume of 32x32x32 voxels. Preprocessing further included cropping to the foreground and z-score normalization.

## Enabling GPU Support

To use GPU resources through Colab, change the runtime to GPU:

    From the "Runtime" menu select "Change Runtime Type"
    Choose "GPU" from the drop-down menu
    Click "SAVE"

This will reset the notebook and probably ask you if you are a robot (these instructions assume you are not). Running

!nvidia-smi

in a cell will verify this has worked and show you what kind of hardware you have access to.


First, we download the data and install some required packages. We will use MONAI and PyTorch.


In [ ]:
!wget https://www.dropbox.com/s/c15o4fi5qjakeym/ixi_t1_fgcrop_32.h5

In [ ]:
!pip install -qU "monai[ignite, nibabel, torchvision, tqdm]==0.6.0"
!pip install -qU "pymia"
!pip install -qU "torchio"

In [ ]:
!nvidia-smi

## Data Loading
The data we use comes in the hdf5 format. While we could also just load the individual images, this is more efficient. Data handling often is a bottleneck in the pipeline.

To handle this dataset, we need some auxiliaries:

In [ ]:
import h5py
import torch
import numpy as np


class H5Dataset(torch.utils.data.Dataset):

    def __init__(self, h5_path, transform=None):
        super(H5Dataset, self).__init__()
        h5_file = h5py.File(h5_path, 'r')
        self.images = h5_file['data']['images']
        self.labels = h5_file['data']['gt']
        self.name = h5_file['meta']['subjects']

    def __getitem__(self, index):
        index_raw = int(index)
        index = str(index).zfill(3)
        img = torch.from_numpy(self.images[index][:]).float()
        label = torch.from_numpy(self.labels[index][:]).to(torch.int64) + 1
        subjects = self.name[index_raw][:].decode('utf-8')
        return {"images": img,
                "labels": label,
                "subjects": subjects}

    def __len__(self):
        return self.length


## The neural network

Current state of the art networks probably have too many learnable parameters for this rather simple task. So we define a simpler network for this classification. It consists of 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, 3, 1, 1),
            nn.InstanceNorm3d(out_channels),
            nn.PReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv3d(out_channels, out_channels, 3, 1, 1),
            nn.InstanceNorm3d(out_channels),
            nn.PReLU(),
        )

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.ones_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.ones_(m.bias)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = F.avg_pool3d(x, 2)
        return x


class Classifier(nn.Module):
    def __init__(self, n_classes, n_input_channels):
        super().__init__()

        self.conv0 = nn.Sequential(
            nn.Conv3d(in_channels=n_input_channels, out_channels=32, kernel_size=7, padding=3, stride=2),
            nn.InstanceNorm3d(32),
            nn.PReLU(),
            nn.MaxPool3d(kernel_size=3, stride=2, padding=1),
        )
        self.featextractor = nn.Sequential(
            ConvBlock(in_channels=32, out_channels=64),
            ConvBlock(in_channels=64, out_channels=128),
            ConvBlock(in_channels=128, out_channels=256),
            nn.AdaptiveAvgPool3d((1, 1, 1))
        )

        self.drop_layer = nn.Dropout(p=0.2)

        self.fc = nn.Linear(256, n_classes)


    def forward(self, x):
        x = self.conv0(x)
        x = self.featextractor(x)
        x = x.view(x.size(0), -1)
        x = self.drop_layer(x)
        out = self.fc(x)

        return out

Next, we load the dataset and randomly create a train/test split. Please note, that we would create train/validation/test splits for a real application, but omit this for the [sake of simplicity](https://xkcd.com/2587/). We reserve 25% of the data for testing.

In [ ]:
dataset = H5Dataset("/content/ixi_t1_fgcrop_32.h5")

In [ ]:
print(len(dataset.images))

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

subjectlist = np.arange(len(dataset.images))
# (dataset.images.keys())
print(list(dataset.images.keys()))

# Reserve 20% of the data for testing
subj_train, subj_test = train_test_split(subjectlist, test_size=0.25, 
                                         shuffle=True, random_state=42)

trainingset = torch.utils.data.Subset(dataset, subj_train)
testset = torch.utils.data.Subset(dataset, subj_test)

## Data loading and augmentations
The very high number of parameters in common neural networks can lead to overfitting. To make this harder, we perturb the training data. This is called "data augmentation". In this example, we randomly flip the image, transform it with an affine matrix, add a bias field, noise and blurring.

In [ ]:
import torchio as tio

import pymia.data.transformation as tfm
import pymia.data.definition as defs


class TorchIOTransform(tfm.Transform):
    """Example wrapper for `TorchIO <https://github.com/fepegar/torchio>`_ transformations."""

    def __init__(self, transforms: list, entries=(defs.KEY_IMAGES, defs.KEY_LABELS)) -> None:
        super().__init__()
        self.transforms = transforms
        self.entries = entries

    def __call__(self, sample: dict) -> dict:
        # unsqueeze samples to be 4-D tensors, as required by TorchIO
        for entry in self.entries:
            if entry not in sample:
                if tfm.raise_error_if_entry_not_extracted:
                    raise ValueError(tfm.ENTRY_NOT_EXTRACTED_ERR_MSG.format(entry))
                continue

            np_entry = tfm.check_and_return(sample[entry], np.ndarray)
            sample[entry] = np.expand_dims(np_entry, -1)

        # apply TorchIO transforms
        for t in self.transforms:
            sample = t(sample)

        # squeeze samples back to original format
        for entry in self.entries:
            np_entry = tfm.check_and_return(sample[entry].numpy(), np.ndarray)
            sample[entry] = np_entry.squeeze(-1)

        return sample

transforms_augmentation = [TorchIOTransform(
    [tio.RandomFlip(axes='LR', flip_probability=0.5), 
     tio.RandomFlip(axes='AP', flip_probability=0.5),
     tio.RandomAffine(scales=(0.85, 1.15), degrees=10, 
                      isotropic=False, default_pad_value='otsu',
                      image_interpolation='NEAREST'),
     tio.RandomBiasField(),
     tio.RandomNoise(),
     tio.RandomBlur(),
     ])]

batchsize = 566

trainloader = torch.utils.data.DataLoader(trainingset, batch_size=batchsize, 
                                          shuffle=True, num_workers=2,
                                          pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batchsize, 
                                         shuffle=False, num_workers=2, 
                                         pin_memory=True)

Let's look at the classification model we created:

In [ ]:
from torchsummary import summary
assert torch.cuda.is_available(), "GPU not available"
device = torch.device("cuda:0")
model = Classifier(n_classes=2, n_input_channels=1)
print(model)
# summary(model, (1, 32, 32, 32))
model.to(device)
# model = Classifier(n_classes=2, n_input_channels=1).to(device).float()

# summary(model, (1, 32, 32, 32))
# print(model)

## The loss and optimizer

Next, we need a loss fuction and an optimizer. We will use the cross-entropy loss and the "Adam" optimizer.

In [ ]:
learning_rate = 1e-4

classificationloss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

## Training
We have everything set up, from the data to the network, so we can finally train our model.

In [ ]:
from monai.metrics import ROCAUCMetric, ConfusionMatrixMetric
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, \
    roc_auc_score

epoch_num = 100
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = list()
metric_values = list()
auc_metric = ROCAUCMetric()
accuracy = list()
train_loss = list()
val_loss = list()
train_accuracy = list()
val_accuracy = list()

for epoch in range(epoch_num):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{epoch_num}")

    epoch_loss = 0
    epoch_loss_val = 0
    step = 1

    steps_per_epoch = len(trainingset) // trainloader.batch_size
    pred_values = list()
    label_values = list()
    pred_values_test = list()
    label_values_test = list()

    # put the network in train mode; this tells the network and its modules to
    # enable training elements such as normalisation and dropout, where applicable
    model.train()
    for batch in trainloader:

      inputs, labels = batch["images"].swapaxes(1, -1).cuda(), batch["labels"].squeeze(1).cuda()

      # prepare the gradients for this step's back propagation
      optimizer.zero_grad()
        
      # run the network forwards
      outputs = model(inputs)
        
      # run the loss function on the outputs
      loss = classificationloss(outputs, labels)
        
      # compute the gradients
      loss.backward()
        
      # tell the optimizer to update the weights according to the gradients
      # and its internal optimisation strategy
      optimizer.step()

      label_values += labels.cpu()
      pred_values += outputs.argmax(dim=1).cpu()

      epoch_loss += loss.item()
      print(f"{step}/{len(trainingset) // trainloader.batch_size + 1}, training_loss: {loss.item():.4f}")
      step += 1

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    train_accuracy.append(accuracy_score(label_values, pred_values))
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
    print("Accuracy: " + str(accuracy_score(label_values, pred_values)))
    
        # switch off training features of the network for this pass
    
    model.eval()

    # 'with torch.no_grad()' switches off gradient calculation for the scope of its context
    with torch.no_grad():
        # create lists to which we will concatenate the the validation results


        val_step = 0
        # iterate over each batch of images and run them through the network in evaluation mode
        for val_data in testloader:
            val_images, val_labels = batch["images"].swapaxes(1, -1).cuda(), batch["labels"].squeeze(1).cuda()

            # run the network
            val_out = model(val_images)

            test_loss = classificationloss(val_out, val_labels)
            epoch_loss_val += test_loss.item()

            pred_values_test += val_out.argmax(dim=1).cpu()
            label_values_test += val_labels.cpu()
            val_step += 1

        val_accuracy.append(accuracy_score(label_values_test, pred_values_test))
        epoch_loss_val /= val_step
        val_loss.append(epoch_loss_val)           

print("Done :-)")
       

Now we can check how the loss and the accuracy evolves over the training process:

In [ ]:
import matplotlib.pyplot as plt

epochs = np.arange(1, len(epoch_loss_values) +1)

plt.figure("train/test", (12, 6))

plt.subplot(2, 2, 1)
plt.title("Training loss")
plt.xlabel("epoch")
plt.plot(epochs, epoch_loss_values)

plt.subplot(2, 2, 2)
plt.title("Test loss")
plt.xlabel("epoch")
plt.plot(epochs, val_loss)

plt.subplot(2, 2, 3)
plt.title("Training accuracy")
y = train_accuracy
plt.xlabel("epoch")
plt.plot(epochs, train_accuracy)

plt.subplot(2, 2, 4)
plt.title("Test accuracy")
plt.xlabel("epoch")
plt.plot(epochs, val_accuracy)
plt.suptitle("Sex classification IXI dataset")
plt.show()